In [132]:
import pandas as pd
import numpy as np
import random
import string
import warnings
warnings.filterwarnings("ignore")

np.random.seed(42)
random.seed(42)

# About

I'm not the biggest fan of Pandas (rather use PySpark), but I'm speed running through all the concepts (via exercises) I think that are important to know for roles that require its usage.

# **Exercise 1 Basic DataFrame Creation**

   - Create a DataFrame with 10 rows and 3 columns: 'A', 'B', and 'C'. Populate column 'A' with integers from 0 to 9, column 'B' with random floats, and column 'C' with random strings. 


## Exercise 1 Approach 1

In [133]:
# This approach uses Python's random choices function to simplify random string generation
data_1 = {
    'A': range(10),
    'B': np.random.rand(10), # Integers from 0 to 9
    'C': [''.join(random.choices(string.ascii_letters, k=5)) for _ in range(10)] # random strings of 5 characters
}

# Create the DataFram
df_1 = pd.DataFrame(data_1)

In [134]:
df_1

,A,B,C
0,0,0.374540,HbolM
1,1,0.950714,JUevb
2,2,0.731994,lAbkH
3,3,0.598658,ClEQa
4,4,0.156019,PKriX
5,5,0.155995,refSF
6,6,0.058084,PLBYt
7,7,0.866176,CRGSE
8,8,0.601115,Kclpe
9,9,0.708073,mfoHs


## Exercise 1 Approach 2

In [135]:
# Approach that uses more NumPy
letters = np.array(list(string.ascii_letters))
random_strings = np.random.choice(letters, size=(10, 5))
random_strings = [''.join(row) for row in random_strings]

data_2 = {
    'A': np.arange(10),
    'B': np.random.rand(10),
    'C': random_strings
}

df_2 = pd.DataFrame(data_2)

In [136]:
df_2

,A,B,C
0,0,0.662522,vbxRD
1,1,0.311711,LbuGl
2,2,0.520068,vRyWA
3,3,0.546710,PBpoU
4,4,0.184854,YRZcK
5,5,0.969585,YguiM
6,6,0.775133,rdynX
7,7,0.939499,izbtB
8,8,0.894827,UgRhU
9,9,0.597900,InqJX


# **Exercise 2 Filtering Data**

   - Create a DataFrame with columns 'Name', 'Age', and 'Score'. Filter out the rows where the 'Age' is less than 20 and 'Score' is greater than 70.


In [137]:
data_3 = {
    'Name': ['Alice', 'Bob', 'Charlie', 'David', 'Eva'],
    'Age': [25, 17, 22, 19, 30],
    'Score': [85, 60, 75, 90, 65]
}

df_3 = pd.DataFrame(data_3)

filtered_df = df_3[(df_3['Age'] >= 20) & (df_3['Score'] > 70)]

In [138]:
filtered_df

,Name,Age,Score
0,Alice,25,85
2,Charlie,22,75


# **Exercise 3 Handling Missing Data**

   - Create a DataFrame with some missing values. Fill missing values in a specific column using the mean of that column, and for another column, forward-fill the missing values.


In [146]:
# Creating a DataFrame with missing values
data_with_missing_values = {
    'Name': ['Alice', 'Bob', 'Charlie', 'David', 'Eva'],
    'Age': [25, np.nan, 22, 19, np.nan],
    'Score': [85, 60, np.nan, 90, 65]
}

def reset_df(data):
    return pd.DataFrame(data)

df_with_missing = reset_df(data_with_missing_values)
df_with_missing

,Name,Age,Score
0,Alice,25.0,85.0
1,Bob,NaN,60.0
2,Charlie,22.0,NaN
3,David,19.0,90.0
4,Eva,NaN,65.0


## **Methods Available**

There are 3 primary ways in which data imputation can be done:

### **Pandas Approach (DataFrame-centric)**
    `fillna()`: Pandas provides a powerful method, `fillna()` that allows you to fill missing values based on different stragtegies (e.g., mean, forward-fill, backward-fill)

   Example

In [147]:
# df_with_mising['Age'].fillna(df_with_mising['Age'].mean(), inplace=True) # inplace=True will...
df_with_missing['Age'].fillna(df_with_missing['Age'].mean(), inplace=True)
df_with_missing['Score'].fillna(method='ffill', inplace=True)

In [148]:
df_with_missing

,Name,Age,Score
0,Alice,25.0,85.0
1,Bob,22.0,60.0
2,Charlie,22.0,60.0
3,David,19.0,90.0
4,Eva,22.0,65.0


### Important Notes

Observe that with this approach, we mutate the original dataframe that had missing values. This could be an issue when it comes to debugging, especially in a pipeline of dataframe transformations. We can address this by setting `inplace=True`. Doing so will result in `fillna()` returning either a Pandas Series (in the case that we modify a single column) or a Pandas Dataframe (in the case that we fill nans across the entire DataFrame).

There are some trade-offs that need to be considered here though. While we gain increased debugging capabilities, we take on increased space consumption for the copies made.

#### Space Complexity Increase for Making a Copy

**(`inplace=True`)**
No new DataFrame is created, and modifications happen in place, so the memory overhead is minimal. This was what we originally tried.

**`inplace=False`**
A new DataFrame (or Series) is created with the modified values, which involves copying the data. The memory overhead depend onhow much of the data is being copied.

- The space consumption for the Series case (modifying one column) would be $O(m)$, the same as the time complexity.
- The space consumption for the DataFrame case (modfing potentially all cells) would be $O(m * n)$, the same as its respective time complexity.




### More Important Notes
So why would we even want Series when we have DataFrames? It seems like this splitting of data types can lead to lots of
confusion within the library, especially with people new to using Pandas.

#### Dimensionality Argument

- A series is essentially a one-dimensional array (like a ist or column) with an index. It represents a single column or row of data in a DataFrame.
- A DataFrame is a two-dimensional table (like a matrix or a spreadsheet), consisting of _multiple_ Series (columns), each potentially of a different type (e.g. integer, float, string)

Having both helps us organize data at different levels of granularity:

- A **Series** holds single-variable data, like a column from a table.
- A **DataFrame** holds multi-variable data, like the full table.

#### Simplicity

Working with single-dimensional data using a **Series** can make code simpler and clearner. We don't have to worry about extra dimensions. A good example of this can be when you need for select a single column to perform operations on like summing, averaging, or filtering.

#### Alignment & Broadcasting

This, in my opinion, is arguably the most important reason for why both exist. Broadcasting operations across rows or columns in a DataFrame. Series allow for alignment based on index, which can simplify combining and manipulating data. See the example in the next set of cells

In [142]:
s = pd.Series([1, 2, 3], index=['a', 'b', 'c'])
df = pd.DataFrame({'A': [10, 20, 30]}, index=['a', 'b', 'c'])

In [143]:
s

a    1
b    2
c    3
dtype: int64

In [144]:
df

,A
a,10
b,20
c,30


In [145]:
# Automatically aligns based on index
result = df['A'] + s
print(result)
type(result)

a    11
b    22
c    33
dtype: int64


pandas.core.series.Series

### **NumPy Approach (Array-centric)**
The second approach to exercise 3 leverages NumPy to perform manual operations, like calculating the mean and applying the forward-fill for the missing values.

In [159]:
# Reset the df
df_with_missing = reset_df(data_with_missing_values)

# Fill missing values in 'Age' using Numpy's mean. --It's just a fancy version of a ternary expression i.e.
# np.where(condition, x, y) is the same as thing = condition? x : y
df_with_missing['Age'] = np.where(pd.isnull(df_with_missing['Age']), np.nanmean(df_with_missing['Age']), df_with_missing['Age'])
pred = pd.isnull(df_with_missing['Score'])
x = df_with_missing['Score'].ffill()
y = df_with_missing['Score']
df_with_missing['Score'] = np.where(pred, x, y)
df_with_missing

,Name,Age,Score
0,Alice,25.0,85.0
1,Bob,22.0,60.0
2,Charlie,22.0,60.0
3,David,19.0,90.0
4,Eva,22.0,65.0


### **Sci-kit-learn Imputation (Machine Learning Approach)**
`SimpleImputer` from `sklearn` will offer strategies like mean, median, most frequent, or custom values to fill missing data. Its by far the most useful for when you need to construct an ML pipeline.

In [163]:
from sklearn.impute import SimpleImputer

df_with_missing = reset_df(data_with_missing_values)

imputer = SimpleImputer(strategy='mean')
df_with_missing['Age'] = imputer.fit_transform(df_with_missing[['Age']])

# NO wo
def forward_fill(data):
    for i in range(1, len(data)):
        if np.isnan(data[i]):
            data[i] = data[i - 1]
    return data

df_with_missing['Score'] = forward_fill(df_with_missing['Score'].to_numpy())

df_with_missing

,Name,Age,Score
0,Alice,25.0,85.0
1,Bob,22.0,60.0
2,Charlie,22.0,60.0
3,David,19.0,90.0
4,Eva,22.0,65.0


### Important Notes

Its important to note that for using forward fill, if the first value is empty, then there will be no previous value for which to fill the nan with and it will remain nan.

# **Remaining Practice to Complete**

### 4. **GroupBy Operation**

   - Given a DataFrame with columns 'City', 'Date', and 'Temperature', group the data by 'City' and find the mean temperature for each city.


### 5. **Apply Function**

   - Create a DataFrame with a 'Salary' column and a 'Years of Experience' column. Use the `.apply()` function to create a new column called 'Salary per Year of Experience'.


### 6. **Pivot Table**

   - Create a DataFrame with columns 'Product', 'Region', 'Sales', and 'Month'. Create a pivot table that shows the sum of 'Sales' for each 'Product' and 'Region' across all 'Months'.


### 7. **Merging DataFrames**

   - You have two DataFrames: one with columns 'StudentID', 'Name', and 'Major', and another with columns 'StudentID' and 'GPA'. Merge the two DataFrames on 'StudentID'.


### 8. **Sorting Data**

   - Create a DataFrame with columns 'Employee', 'Department', and 'Salary'. Sort the DataFrame first by 'Department', and within each department, by 'Salary' in descending order.


### 9. **Date Manipulation**

   - Create a DataFrame with a 'Date' column containing a range of dates (e.g., from January 1, 2021, to January 10, 2021). Add a column 'Day of the Week' that contains the day of the week for each date.


### 10. **String Manipulation**

   - Create a DataFrame with a 'Text' column that contains strings with mixed cases and leading/trailing spaces. Clean the data by trimming the spaces and converting all the text to lowercase.


### 11. **Rolling Window**

   - Given a DataFrame with a 'Date' and 'Stock Price' column, compute the 3-day moving average for the stock prices.


### 12. **Categorical Data**

   - Create a DataFrame with a 'Fruit' column that contains repeated entries (e.g., 'Apple', 'Banana', 'Orange'). Convert the 'Fruit' column to a categorical data type and assign custom categories in the order: 'Banana', 'Orange', 'Apple'.


### 13. **Multi-Index**

   - Create a DataFrame with a multi-level index. The first index should be 'Country' and the second 'Year'. The data should include columns like 'Population' and 'GDP'. Access data for a specific country and year.


### 14. **Cumulative Sum**

   - Create a DataFrame with a 'Sales' column and compute the cumulative sum of sales over time.


### 15. **Unique Values**

   - Create a DataFrame with a 'Color' column that contains repeated colors. Find the unique colors and count the number of occurrences of each color.


### 16. **Plotting Data**

   - Create a DataFrame with columns 'Year' and 'Sales'. Plot the sales data over time using Pandas' built-in plotting functionality.


### 17. **Data Aggregation**

   - Given a DataFrame with columns 'Team', 'Player', and 'Score', group the data by 'Team' and aggregate the scores (e.g., sum, mean, or max).


### 18. **Sampling Data**

   - Create a DataFrame with 100 rows of random data. Use Pandas' `sample()` method to randomly select 10 rows from the DataFrame.


### 19. **Concatenation**

   - You have two DataFrames: one with columns 'ID' and 'Name', and another with columns 'ID' and 'Address'. Concatenate the DataFrames along the columns.


### 20. **Exploratory Data Analysis**

   - Load a toy dataset (e.g., from Pandas' built-in datasets or from a CSV). Perform basic exploratory analysis by calculating descriptive statistics (e.g., mean, median, standard deviation), handling missing data, and generating a few basic plots.